# Purpose

This notebook analyzes statistics for DOE SETO staff using project portfolio data. For example, average number of projects per staff member is one metric that may be counted here.

## Data

The data used herein are from the Solar Information Management System (SIMS) and the code is written using the assumed structure of the SIMS data XLSX export as of 3-7-2019. If the system evolves substantially in terms of its data exporting between this date and when you're reading this notebook, changes to the code will be necessary.

## Objectives

1. Aggregate projects by TM Prime
    1. Will need to look at unique TM Prime name values first, make sure we don't have any lists of names (and pick one from the list in that scenario)
2. Count number of unique projects per TM Prime, then average that number
3. Plot number of projects per TM Prime by TM Prime and aggregate up to subprogram
4. Average, at TM Prime aggregation level, the amount of Federal dollars, cost share dollars, and total dollars to get a metric of "this many dollars managed per staffer".
5. Plot dollars per TM Prime

In [ ]:
#Make sure any changes to custom packages can be reflected immediately 
#in the notebook without kernel restart

import autoreload
%load_ext autoreload
%autoreload 2

In [ ]:
# Standard plotly imports
import plotly.plotly as py
import plotly.graph_objs as go
from plotly.offline import iplot, init_notebook_mode

# Using plotly + cufflinks in offline mode
import cufflinks
cufflinks.go_offline(connected=True)
init_notebook_mode(connected=True)

import pandas as pd

In [ ]:
pd.options.display.max_columns = 50
pd.options.display.max_rows = 30

In [ ]:
df = pd.read_excel(r'data/Staff_Projects_Money_3-5-19.xlsx', sheet_name = 'Sheet')
df

In [ ]:
df.columns

In [ ]:
df.rename(columns = {'Planned DOE Share ($)': 'Federal Share', 
                     'Planned C/S ($)': 'Cost Share',
                    'Planned Total Cost': 'Total Cost'}, inplace = True)

In [ ]:
df.columns

### Let's see what the distribution of federal funds looks like

In [ ]:
df['Federal Share'].iplot(kind='hist', xTitle='Federal $$',
                        yTitle='Count', title='Distribution of Project Federal Funds')

### OK, time to check out the staff info

In [ ]:
df['TM Prime'].unique()

In [ ]:
df[df['TM Prime'].str.contains('; ', na = False)]

**It turns out that there are a few rows in which there's a semicolon-delimited list of Prime TMs given. Leveraging my own knowledge of these projects, it looks like later values in that list are the more recent correct values. So I'll default to pulling them out as the replacements.**

In [ ]:
#Can't seem to get pd.Series.replace() to work, so have to do this manually unfortunately...

df.loc[45, 'TM Prime'] = 'David Walter'
df.loc[65, 'TM Prime'] = 'Ammar Qusaibaty'
df.loc[90, 'TM Prime'] = 'Mark Lausten'
df.loc[93, 'TM Prime'] = 'Inna Kozinsky'
df.loc[121, 'TM Prime'] = 'Dave Rench McCauley'
df.loc[231, 'TM Prime'] = 'David Walter'
df.loc[269, 'TM Prime'] = 'Vijay Rajgopal'

In [ ]:
df[df['TM Prime'].str.contains('; ', na = False)]

In [ ]:
df['TM Prime'].value_counts()

In [ ]:
pd.Series(df['TM Prime'].unique()).sort_values()

In [ ]:
pd.Series(df['TM Prime'].unique())[pd.Series(df['TM Prime'].unique()).str.contains("  ", na = False)]

**Whoa! What's going on here? It looks like some people have 2 spaces between their first and last name, instead of one, but they also have lots of records in which it's just one space. I'll have to correct these.**

In [ ]:
df['TM Prime'].replace({'Andrew  Dawson': 'Andrew Dawson',
                       'Marie  Mapes': 'Marie Mapes',
                       'Lenny  Tinker': 'Lenny Tinker',
                       'Michele  Boyd': 'Michele Boyd',
                       'Tassos  Golnas': 'Tassos Golnas',
                       'Dave  Rench McCauley': 'Dave Rench McCauley',
                       'Susan  Huang': 'Susan Huang',
                       'Ammar  Qusaibaty': 'Ammar Qusaibaty',
                       'Andru  Prescod': 'Andru Prescod',
                       'Christian  Philipsen': 'Christian Philipsen'}, inplace = True)

In [ ]:
pd.Series(df['TM Prime'].unique()).sort_values().values

**That did the trick!**

In [ ]:
df.columns

### Projects and Dollars Per Staffer

Here we'll aggregate by TM Prime to figure out how many awards each person has and how much money they manage, on average.

In [ ]:
df.groupby(['TM Prime'])['Federal Share', 'Cost Share', 'Total Cost'].count()

In [ ]:
df.groupby(['TM Prime'])['Federal Share', 'Cost Share', 'Total Cost'].count().sum()

In [ ]:
df.groupby(['TM Prime'])['Federal Share', 'Cost Share', 'Total Cost'].count().describe()

In [ ]:
df.groupby(['TM Prime'])['Federal Share', 'Cost Share', 'Total Cost'].sum()

In [ ]:
df.groupby(['TM Prime'])['Federal Share', 'Cost Share', 'Total Cost'].sum().sum()

In [ ]:
df.groupby(['TM Prime'])['Federal Share', 'Cost Share', 'Total Cost'].sum().describe()